In [5]:
import psycopg2
import geopandas as gpd
import folium
import fiona

In [15]:
query = """

 WITH aois AS (WITH hulls AS (
 WITH hulls AS (
         WITH clusters AS (
         WITH preclusters_subset AS (
         SELECT * FROM preclusters WHERE st_intersects(hull, 
     (SELECT ST_Buffer(ST_GeomFromText('POINT(950779.7708653511 6003006.738011813)', 3857), 1000) AS bbox)
         )
         )
         SELECT preclusters_subset.id AS precluster_id,
                geometry,
                ST_ClusterDBSCAN(geometry, eps := 35, minpoints := preclusters_subset.dbscan_minpts) over () AS cid
         FROM pois, preclusters_subset
         WHERE ST_Within(geometry, preclusters_subset.hull)
         )
         SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS geometry
         FROM clusters
         WHERE cid IS NOT NULL
         GROUP BY cid
         ),
 intersecting_lines AS (
     SELECT hulls.cid, ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls
     WHERE osm_id = ANY(
       SELECT id FROM planet_osm_ways
       WHERE nodes && ARRAY[5276753769, 4324863920, 5276753768, 4324863909, 30826463, 30795605, 30795604, 211185000, 5257661207, 5257661211, 30795581, 2554281733, 598569841, 3251966772, 30795580, 5257661208, 2554281748, 211185009, 2554281730, 3813829087, 1137992200, 211184998, 3779191473, 2554281735, 598569840, 3028549105, 300837214, 2554281739, 26889117, 3947490897, 34505894, 5629311355, 140403043, 30826465, 34505879, 3919338896, 34505893, 3919338900, 3779191470, 3770972265, 3947490896, 300836808, 5619163221, 92485068, 635164798, 2528311035, 2275605562, 140402611, 60022499, 3919338897, 1552956916, 3947490895, 156358794, 1552956921, 1552918950, 1821566786, 1552956940, 1552956933, 27481920, 1552956926, 3779191466, 5116225726, 3770972267, 5629311354, 316405255, 5679622650, 5116225264, 5257661186, 140401147, 4917184482, 3437355089, 4741926255, 277971968, 29033205, 4333509523, 5682247773, 2884057358, 2884057363, 3964600251, 4333509524, 2884057353, 5285898002, 4324863901, 27481860, 288532121, 4920833565, 2027117410, 4920833571, 3931738076, 4324863909, 28582322, 28137435, 2015772140, 28137423, 5283725456, 28582095, 27256989, 5682255962, 28587330, 1584034537, 702999103, 28137436, 5680499464, 28584142, 2957062050, 2567818929, 2957061242, 2957062051, 2645535381, 5283725090, 1584034534, 2567854590, 1584034520, 5283725091, 318214542, 28584110, 2957062053, 28582116, 5297597984, 28582192, 936942843, 2567818138, 279263027, 309916559, 5682256160, 5283725115, 5283725105, 5283725103, 443221965, 1372024222, 29382417, 71370877, 103831747, 3941436225, 27481777, 4919284831, 3908987596, 4919284832, 4919284833, 4919284829, 1377025805, 103831661, 1387934390, 1373290806, 27488750, 1545781170, 1739247172, 4963453607, 4963453578, 1491966868, 2723918992, 147659457, 4963453600, 2723918988, 4963453576, 4963456228, 4963453608, 1747362672, 4963453596, 5397966255, 1747362679, 5232517237, 3071329228, 3071329230, 3832856564, 3071329232, 1923986931, 30230049, 3071329227, 318314466, 5127862287, 443219909, 3813718388, 4333456496, 27256940, 3813718391, 5047325371, 3813718387, 364712760, 1969942776, 364712757, 1600555947, 5038554591, 1971833130, 5038554592, 4919284845, 3908987593, 1708877385, 27481777, 1540203133, 4919284504, 3941436434, 4919284833, 5038554574, 3829552476, 5038554588, 5038554575, 1037698619, 27462669, 4333534664, 539248401, 3777963165, 1670336126, 5321971794, 26795404, 3907499417, 5321971774, 1217460486, 5321971771, 5321971803, 3017726326, 1640256896, 5321971790, 4787197078, 26687706, 5321972808, 445397477, 1396809768, 1396809788, 29055496, 300391057, 445400037, 5321971791, 445397706, 5183760625, 5321971777, 3098370972, 26114459, 2884057376, 4333509530, 527010463, 4333509529, 5285898010, 2884057378, 1638473930, 1638473919, 275342452, 5297695190, 1638473924, 1638473928, 2514503529, 1373290806, 30825907, 2884057398, 1572922678, 2277024770, 2277024776, 5255883986, 5255883988, 89010597, 5257663483, 5255883985, 5255884011, 5255883989, 150535753, 150535745, 27256990, 28582171, 27256991, 631160397, 2957061238, 5283725438, 107197962, 1787928994, 1036170675, 4598564401, 1787929110, 1493689564, 1941582705, 27481860, 5403574214, 5403574213, 1584068303, 5080924278, 1493689558, 4919284516, 3908987593, 994390114, 27481783, 3941433590, 4919284510, 4919284514, 225972336, 5682256010, 28581993, 5285897992, 2172694607, 5285898014, 107190635, 5682256009, 312220302, 5682256019, 5682247785, 30826182, 5285897991, 107190633, 5285897994, 312220299, 30826181, 300028688, 5115701914, 26114490, 1027163102, 1027163115, 300028168, 1027200899, 5219957759, 28887288, 26114491, 3813718398, 3813829086, 696114760, 29033242, 1027163117, 1027200723, 29033251, 3779243541, 1027201096, 1586370783, 2959909849, 1498834449, 1498834453, 1964156630, 1498834448, 1498834440, 703217716, 1498834441, 3973971896, 1517807767, 1517807736, 5403574200, 1498834439, 5403574186, 5403574187, 1517807752, 1586370817, 5403574202, 1498834437, 1600202209, 2886648425, 1498834451, 5164655286, 1517807741, 1498834447, 1964156632, 111413878, 1517807749]::bigint[]
     )
     AND ST_DWithin(planet_osm_line.way, hulls.geometry, 50)
 )
 
 SELECT 1 AS color, ST_ConcaveHull(ST_Union(geometry), 0.99) AS geometry FROM (
   SELECT cid, geometry FROM hulls
   UNION
   SELECT cid, geometry FROM intersecting_lines
 ) AS tmp
 GROUP BY cid
 )
 SELECT ST_Difference(hulls.geometry, coalesce((
     SELECT ST_Union(way) AS geometry FROM planet_osm_polygon
     WHERE (water IS NOT NULL OR waterway IS NOT NULL)
           AND (tunnel IS NULL OR tunnel = 'no')
     AND st_intersects(way, hulls.geometry)
 ), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
 FROM hulls)
 SELECT (ST_Dump(ST_Union(geometry))).geom AS geometry FROM aois
       

"""

In [16]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(query, conn, geom_col='geometry')
    
aois.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.3768, 8.5432], zoom_start=15)
folium.GeoJson(aois).add_to(m)
m

In [49]:
query = """

 WITH aois AS (WITH hulls AS (
 WITH hulls AS (
         WITH clusters AS (
         WITH preclusters_subset AS (
         SELECT * FROM preclusters WHERE st_intersects(hull, 
     (SELECT ST_Buffer(ST_GeomFromText('POINT(950779.7708653511 6003006.738011813)', 3857), 1000) AS bbox)
         )
         )
         SELECT preclusters_subset.id AS precluster_id,
                geometry,
                ST_ClusterDBSCAN(geometry, eps := 35, minpoints := preclusters_subset.dbscan_minpts) over () AS cid
         FROM pois, preclusters_subset
         WHERE ST_Within(geometry, preclusters_subset.hull)
         )
         SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS geometry
         FROM clusters
         WHERE cid IS NOT NULL
         GROUP BY cid
         ),
 intersecting_lines AS (
     SELECT hulls.cid, ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls
     WHERE osm_id = ANY(
       SELECT id FROM planet_osm_ways
       WHERE nodes && ARRAY[5276753769, 4324863920, 5276753768, 4324863909, 30826463, 30795605, 30795604, 211185000, 5257661207, 5257661211, 30795581, 2554281733, 598569841, 3251966772, 30795580, 5257661208, 2554281748, 211185009, 2554281730, 3813829087, 1137992200, 211184998, 3779191473, 2554281735, 598569840, 3028549105, 300837214, 2554281739, 26889117, 3947490897, 34505894, 5629311355, 140403043, 30826465, 34505879, 3919338896, 34505893, 3919338900, 3779191470, 3770972265, 3947490896, 300836808, 5619163221, 92485068, 635164798, 2528311035, 2275605562, 140402611, 60022499, 3919338897, 1552956916, 3947490895, 156358794, 1552956921, 1552918950, 1821566786, 1552956940, 1552956933, 27481920, 1552956926, 3779191466, 5116225726, 3770972267, 5629311354, 316405255, 5679622650, 5116225264, 5257661186, 140401147, 4917184482, 3437355089, 4741926255, 277971968, 29033205, 4333509523, 5682247773, 2884057358, 2884057363, 3964600251, 4333509524, 2884057353, 5285898002, 4324863901, 27481860, 288532121, 4920833565, 2027117410, 4920833571, 3931738076, 4324863909, 28582322, 28137435, 2015772140, 28137423, 5283725456, 28582095, 27256989, 5682255962, 28587330, 1584034537, 702999103, 28137436, 5680499464, 28584142, 2957062050, 2567818929, 2957061242, 2957062051, 2645535381, 5283725090, 1584034534, 2567854590, 1584034520, 5283725091, 318214542, 28584110, 2957062053, 28582116, 5297597984, 28582192, 936942843, 2567818138, 279263027, 309916559, 5682256160, 5283725115, 5283725105, 5283725103, 443221965, 1372024222, 29382417, 71370877, 103831747, 3941436225, 27481777, 4919284831, 3908987596, 4919284832, 4919284833, 4919284829, 1377025805, 103831661, 1387934390, 1373290806, 27488750, 1545781170, 1739247172, 4963453607, 4963453578, 1491966868, 2723918992, 147659457, 4963453600, 2723918988, 4963453576, 4963456228, 4963453608, 1747362672, 4963453596, 5397966255, 1747362679, 5232517237, 3071329228, 3071329230, 3832856564, 3071329232, 1923986931, 30230049, 3071329227, 318314466, 5127862287, 443219909, 3813718388, 4333456496, 27256940, 3813718391, 5047325371, 3813718387, 364712760, 1969942776, 364712757, 1600555947, 5038554591, 1971833130, 5038554592, 4919284845, 3908987593, 1708877385, 27481777, 1540203133, 4919284504, 3941436434, 4919284833, 5038554574, 3829552476, 5038554588, 5038554575, 1037698619, 27462669, 4333534664, 539248401, 3777963165, 1670336126, 5321971794, 26795404, 3907499417, 5321971774, 1217460486, 5321971771, 5321971803, 3017726326, 1640256896, 5321971790, 4787197078, 26687706, 5321972808, 445397477, 1396809768, 1396809788, 29055496, 300391057, 445400037, 5321971791, 445397706, 5183760625, 5321971777, 3098370972, 26114459, 2884057376, 4333509530, 527010463, 4333509529, 5285898010, 2884057378, 1638473930, 1638473919, 275342452, 5297695190, 1638473924, 1638473928, 2514503529, 1373290806, 30825907, 2884057398, 1572922678, 2277024770, 2277024776, 5255883986, 5255883988, 89010597, 5257663483, 5255883985, 5255884011, 5255883989, 150535753, 150535745, 27256990, 28582171, 27256991, 631160397, 2957061238, 5283725438, 107197962, 1787928994, 1036170675, 4598564401, 1787929110, 1493689564, 1941582705, 27481860, 5403574214, 5403574213, 1584068303, 5080924278, 1493689558, 4919284516, 3908987593, 994390114, 27481783, 3941433590, 4919284510, 4919284514, 225972336, 5682256010, 28581993, 5285897992, 2172694607, 5285898014, 107190635, 5682256009, 312220302, 5682256019, 5682247785, 30826182, 5285897991, 107190633, 5285897994, 312220299, 30826181, 300028688, 5115701914, 26114490, 1027163102, 1027163115, 300028168, 1027200899, 5219957759, 28887288, 26114491, 3813718398, 3813829086, 696114760, 29033242, 1027163117, 1027200723, 29033251, 3779243541, 1027201096, 1586370783, 2959909849, 1498834449, 1498834453, 1964156630, 1498834448, 1498834440, 703217716, 1498834441, 3973971896, 1517807767, 1517807736, 5403574200, 1498834439, 5403574186, 5403574187, 1517807752, 1586370817, 5403574202, 1498834437, 1600202209, 2886648425, 1498834451, 5164655286, 1517807741, 1498834447, 1964156632, 111413878, 1517807749]::bigint[]
     )
     AND ST_DWithin(planet_osm_line.way, hulls.geometry, 50)
 )
 
 SELECT 1 AS color, ST_ConcaveHull(ST_Union(geometry), 0.99) AS geometry FROM (
   SELECT cid, geometry FROM hulls
   UNION
   SELECT cid, geometry FROM intersecting_lines
 ) AS tmp
 GROUP BY cid
 )
 SELECT ST_Difference(hulls.geometry, coalesce((
     SELECT ST_Union(way) AS geometry FROM planet_osm_polygon
     WHERE (water IS NOT NULL OR waterway IS NOT NULL)
           AND (tunnel IS NULL OR tunnel = 'no')
     AND st_intersects(way, hulls.geometry)
 ), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
 FROM hulls)
 SELECT ST_Simplify((ST_Dump(ST_Union(geometry))).geom, 3) AS geometry FROM aois
       

"""

In [50]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(query, conn, geom_col='geometry')
    
aois.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.3768, 8.5432], zoom_start=15)
folium.GeoJson(aois).add_to(m)
m